# Main analysis file for Vogelstein project

## Get the data

In [2]:
import pandas as pd
import numpy as np
import os
import nibabel as nib

In [8]:
mri1 = nib.load('sub-0025865_ses-1_dwi_aligned.nii')
mri2 = nib.load('sub-0025865_ses-2_dwi_aligned.nii')

In [13]:
mri1.shape

(182, 218, 182, 31)

In [12]:
mri1.shape == mri2.shape

True

## Scale the data

Scale the data (window) to allow easy viewing of the data and plot time=5 and z slice=15 from each session (and explain why you chose the levels that you did)

In [19]:
# TODO: scale data
# TODO: plot t=5, z slice=15 from each session (2 sessions total).
# 2 plots total. Use matplotlib or seaborn for this?

## Crop out a brain area, plot that region for each session

In [16]:
# Hippocampus is always fun
# TODO: figure out best way to plot MRI data.

## Choose an intensity value at slice 15, return number of pixels above that threshold for each session

In [17]:
# TODO: write a loop that outputs this information.
# TODO: Figure out how to determine what threshold to use.
# Assuming 'intensity value' is equivalent to 'threshold'

## Using threshold, perform first-order analysis

In [15]:
# TODO: figure out what 'first-order analysis' means.

## Train a random forest classifier to differentiate between sessions.

### Need out-of-bag error and distribution of cllass conditional posteriors

In [20]:
# TODO: figure out how random forest classifiers work.
# TODO: learn what out-of-bag error means.
# TODO: learn what a class conditional posterior estimated random forest is.